## Data Collection

- Import
- Set working directory
- Get data from Kaggle

# Import packages

In [33]:
! pip install -r /workspace/PP5/requirements.txt

In [34]:
import numpy
import os

# Change working directory

In [35]:
current_dir = os.getcwd()
current_dir

'/workspace/PP5'

In [36]:
os.chdir('/workspace/PP5')
print("You set a new current directory")

You set a new current directory


In [37]:
current_dir = os.getcwd()
current_dir

'/workspace/PP5'

# Install Kaggle

In [38]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


Run the cell below **to change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON**.

In [39]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

chmod: cannot access 'kaggle.json': No such file or directory


Set the Kaggle Dataset and Download it.

In [40]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherryleaves_dataset"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Dataset URL: https://www.kaggle.com/datasets/codeinstitute/cherry-leaves
License(s): unknown
 96%|████████████████████████████████████▌ | 53.0M/55.0M [00:02<00:00, 34.8MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 26.7MB/s]


Unzip the downloaded file, and delete the zip file.

In [41]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# Data Preparation

---

## Data cleaning

### Check and remove non-image files

In [42]:
def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [43]:
remove_non_image_file(my_data_dir='inputs/cherryleaves_dataset/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


## Split train validation test set

In [44]:
import os
import shutil
import random
import joblib


def split_train_validation_test_images(my_data_dir, train_set_ratio, validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio + test_set_ratio should sum to 1.0")
        return

    # gets classes labels
    labels = os.listdir(my_data_dir)  # it should get only the folder name
    if 'test' in labels:
        pass
    else:
        # create train, test folders with classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # move a given file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/' + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # move a given file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/' + file_name)

                else:
                    # move given file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/' + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

 - The training set is divided into a 0.70 ratio of data.
 - The validation set is divided into a 0.10 ratio of data.
 - The test set is divided into a 0.20 ratio of data

In [45]:
split_train_validation_test_images(my_data_dir=f"inputs/cherryleaves_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )

# Resize images

Resizing images in datasets to 100px x 100px for further process.

In [46]:
# Parts of code from Walkthrough project 1 - Code Institute 
# Parts of code from https://stackoverflow.com/questions/48121916/numpy-resize-rescale-image

import cv2

data_dir = 'inputs/cherryleaves_dataset/cherry-leaves'

def image_resize(data_dir, new_size=(30, 30)):
    total_files_processed = 0
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(root, file)
                
                img = cv2.imread(file_path)
                img_resized = cv2.resize(img, dsize=new_size, interpolation=cv2.INTER_CUBIC)
                
                cv2.imwrite(file_path, img_resized)
                
                total_files_processed += 1

    print(f"Processed {total_files_processed} files in {data_dir}")
    print(f"All images resized to {new_size[0]}px x {new_size[1]}px")

# Resize images in the train, validation, and test directories
print("Processing train dataset...")
image_resize(os.path.join(data_dir, 'train'))

print("Processing validation dataset...")
image_resize(os.path.join(data_dir, 'validation'))

print("Processing test dataset...")
image_resize(os.path.join(data_dir, 'test'))

Processing train dataset...
Processed 2944 files in inputs/cherryleaves_dataset/cherry-leaves/train
All images resized to 30px x 30px
Processing validation dataset...
Processed 420 files in inputs/cherryleaves_dataset/cherry-leaves/validation
All images resized to 30px x 30px
Processing test dataset...
Processed 844 files in inputs/cherryleaves_dataset/cherry-leaves/test
All images resized to 30px x 30px
